Step 1: Ingestion and Clause identification.

    User uploads TA. Function splits user uploaded TA into meangingful clauses  

Step 2: Clause Comparison. RAG 1 for Tenancy Agreement

    Vector store: RAG 1 - Ideal Clauses from reference TAs
    Compare and retrieve: Retrieve relevant RAG clauses based on similarity to clauses in user uploaded TA
    LLM evaluation: Use LLM to compare user clause and ideal clauses. Provide feedback on TA through a report

Step 3: Post-Report Q and A from User. RAG 2 for Q and A

    Vector store: RAG 2 - Q and A from Database Requirements.xlsx
    Compare and retrieve: Retrieve relevant RAG Q and A clauses based on similarity to user's question
    LLM evaluation: Use LLM to take in additional context from RAG to improve answer.

Next Step: Template recomendation? Integrating law into RAG? 

    Currently we only compare clauses of our RAG database with uploaded user TA. No way to compare and say what clauses must be in the TA. Maybe we can include that in the prompt, instead of using RAG for this part? Like "Tenancy agreement should have a, b, c, d clauses at minimum and most will also have e, f clauses" Also need to figure out how to integrate legal documents into RAG 1 to help explain legality of certain clauses. Right now it just compares clauses in user TA to clauses in RAG database

In [2]:
import pandas as pd
import numpy as np
import textwrap
import re
# Core LangChain/HuggingFace RAG Imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# SentenceTransformer is replaced by HuggingFaceEmbeddings(model_name=...)
# faiss is replaced by FAISS from langchain_community
# We will still need numpy for the internal dataframe processing
# We will use the already configured SentenceTransformer model name
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
K = 3 # Number of top results to retrieve

# Initialize the shared embedding model once
embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

In [ ]:
# --- RAG 1: Ideal Clauses (The 'Gold Standard' for Comparison) ---
def build_ideal_clauses_retriever(data_directory="./TA_template"):
    """
    Loads, chunks, and indexes the ideal tenancy agreement PDFs (RAG 1 source).
    Returns a LangChain FAISS Retriever.
    """
    print("\n--- BUILDING RAG 1: IDEAL CLAUSES RETRIEVER ---")
    
    # 1. Load Documents
    loader = DirectoryLoader(
        path=data_directory,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    all_documents = loader.load()
    print(f"Loaded {len(all_documents)} document pages.")

    # 2. Chunk Documents (using the custom, structure-aware splitter)
    custom_separators = [
        "\n\n",
        r"\n\s*[A-Z]+\s+\d*\s*\.",
        r"\n\s*\d+\.\d*\s*",
        r"\n\s*\([a-zA-Z0-9]+\)\s*",
        "\n", " ", ""
    ]
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        separators=custom_separators,
        is_separator_regex=True
    )
    all_chunks = text_splitter.split_documents(all_documents)
    print(f"Split into {len(all_chunks)} chunks.")

    print("\n--- IDEAL CLAUSES CHUNK PREVIEW (First 3) ---")
    for i, chunk in enumerate(all_chunks[:3]):
        print(f"Chunk {i+1} (Length: {len(chunk.page_content)}):")
        # Print the first 200 characters to keep the output manageable
        print(f"  {chunk.page_content[:200].replace('\n', ' ')}...") 
        print(f"  Metadata: {chunk.metadata}")
        print("-" * 20)


    # 3. Create Vector Store and Retriever
    vectorstore = FAISS.from_documents(all_chunks, embeddings)
    print("FAISS index for Ideal Clauses created successfully.")
    
    # Return the retriever instance
    return vectorstore.as_retriever(search_kwargs={"k": K})

In [4]:
from langchain_core.documents import Document # Import required for conversion

# --- RAG 2: General Q&A (Excel Source) ---
def build_general_qa_retriever(file_path):
    """
    Loads data from the Excel file, converts it to Documents, and creates a FAISS retriever.
    """
    print("\n--- BUILDING RAG 2: GENERAL Q&A RETRIEVER ---")
    
    # 1. Load and process data (using the existing logic)
    try:
        df = pd.read_excel(file_path, header=1)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Using dummy data.")
        df = pd.DataFrame({
            'Question': ["How do I know if my tenancy agreement is valid?"],
            'Answer / Explanation': ["Must be in writing, signed by both parties, and include essential terms."],
            'Legal Commentary': ["Only agreements containing essential terms are enforceable."],
            'Government Regulation / Explanation': ["N/A"]
        })
    
    # 2. Convert Data Rows into LangChain Documents
    documents = []
    for index, row in df.iterrows():
        content = textwrap.dedent(f"""
            Question: {row.get('Question', 'N/A')}
            Answer/Explanation: {row.get('Answer / Explanation', 'N/A')}
            Legal Context: {row.get('Legal Commentary', 'N/A')}
            Regulation/Source: {row.get('Government Regulation / Explanation', 'N/A')}
        """).strip()
        
        # Create a LangChain Document
        doc = Document(
            page_content=content,
            metadata={"source": file_path, "row_index": index}
        )
        documents.append(doc)
        
    print(f"Created {len(documents)} General Q&A Documents.")

    print("\n--- GENERAL Q&A CHUNK PREVIEW (First 3) ---")
    for i, doc in enumerate(documents[:3]):
        print(f"Chunk {i+1}:")
        print(doc.page_content)
        print(f"  Metadata: {doc.metadata}")
        print("-" * 20)

    # 3. Create Vector Store and Retriever
    vectorstore = FAISS.from_documents(documents, embeddings)
    print("FAISS index for General Q&A created successfully.")
    
    return vectorstore.as_retriever(search_kwargs={"k": K})

In [ ]:
# Testing RAG 1 and 2

# --- Configuration ---
FILE_NAME = 'Database Requirements.xlsx'
DATA_DIRECTORY = "./TA_template" 

# Initialize shared embedding model once
embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

if __name__ == '__main__':
    # 1. BUILD THE TWO RAG SYSTEMS
    ideal_clauses_retriever = build_ideal_clauses_retriever(DATA_DIRECTORY)
    general_qa_retriever = build_general_qa_retriever(FILE_NAME)

    print("\n✅ BOTH RAG SYSTEMS ARE READY.")
    print("--------------------------------------------------")
    
    # --- SIMULATE THE PIPELINE EXECUTION ---
    
    # --- PHASE 1: CLAUSE IDENTIFICATION & COMPARISON (RAG 1) ---
    print("\n--- PHASE 1: COMPARING USER AGREEMENT CLAUSES ---")
    
    # 1.1 Simulate a single clause from the user's uploaded document
    # In the full app, you would run the text_splitter on the user upload here
    user_clause_to_check = "(b) SECURITY DEPOSIT: The Tenant shall pay the Landlord a security deposit equal to $3,000. This deposit will not accrue interest."
    
    # 1.2 Use RAG 1 to retrieve the Ideal Clause context
    comparison_context = ideal_clauses_retriever.invoke(user_clause_to_check)
    
    # 1.3 LLM Call for Feedback (Simulated)
    print(f"\nUser Clause: {user_clause_to_check}")
    print("\n[SIMULATED LLM COMPARISON & FEEDBACK GENERATION]")
    print(f"Retrieved Ideal Context for LLM:")
    for doc in comparison_context:
        print(f"  - Length: {len(doc.page_content)}. Source: {doc.metadata.get('source', 'Unknown')}")
        
    # *A real LLM would now compare the user clause to this context and generate a report.*

    # --- PHASE 2: POST-FEEDBACK Q&A (RAG 2) ---
    print("\n\n--- PHASE 2: GENERAL Q&A ---")
    
    # 2.1 User enters a follow-up question
    user_qa_query = "What rules apply to my landlord entering my rental?"
    
    # 2.2 Use RAG 2 to retrieve the General Q&A context
    qa_context = general_qa_retriever.invoke(user_qa_query)
    
    # 2.3 LLM Call for Answer (Simulated)
    print(f"\nUser Q&A Query: {user_qa_query}")
    print("\n[SIMULATED LLM ANSWER GENERATION]")
    print(f"Retrieved General Q&A Context for LLM:")
    for doc in qa_context:
        print(f"  - Length: {len(doc.page_content)}. Source: {doc.metadata.get('source', 'Unknown')}")
        
    # *A real LLM would now synthesize these into a single, cohesive answer.*


--- BUILDING RAG 1: IDEAL CLAUSES RETRIEVER ---


  0%|          | 0/7 [00:00<?, ?it/s]incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
 29%|██▊       | 2/7 [00:00<00:00, 17.81it/s]Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
 57%|█████▋    | 4/7 [00:00<00:00,  5.63it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
 71%|███████▏  | 5/7 [00:00<00:00,  5.71it/s]Ignoring wr

Loaded 26 document pages.
Split into 108 chunks.

--- IDEAL CLAUSES CHUNK PREVIEW (First 3) ---
Chunk 1 (Length: 903):
  RENTAL AGREEMENT PROBLEMATIC RENTAL TERMS LANDLORD: ABC Property Company TENANT: John Doe RENTAL AMOUNT: S$3,500 per month SECURITY DEPOSIT: S$17,500 (5 months rent) - EXCESSIVE! LEASE PERIOD: 12 mont...
  Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-08-31T22:29:15+08:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-08-31T22:29:15+08:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'TA_data\\TA1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
--------------------
Chunk 2 (Length: 879):
  RESIDENTIAL TENANCY AGREEMENT (Singapore) 1. PARTIES TO THE AGREEMENT LANDLORD: Mr. Tan Ah Beng NRIC: S1234567A Address: 123 Orchard Road, #15-01, Singapore 238123 Contact: +65 9123 4567 TENANT: Ms. S...
  Metadata: {'producer': 'ReportLab PDF Library - 

In [6]:
# Breaking down user uploaded TA

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document # Needed to wrap the user text
from langchain_community.document_loaders import PyPDFLoader
import os # To handle file paths

def load_and_extract_pdf_text(file_path: str) -> str:
    """
    Checks if a file is a PDF, loads it, and extracts all text content.
    
    Args:
        file_path: The local path to the user's uploaded file.

    Returns:
        A single string containing all text from the PDF.

    Raises:
        ValueError: If the file is not found or is not a PDF.
    """
    print(f"\n--- Loading and Extracting Text from: {file_path} ---")

    # 1. Basic File Check
    if not os.path.exists(file_path):
        raise ValueError(f"Error: File not found at path: {file_path}")

    # 2. PDF Extension Check (Simple approach)
    if not file_path.lower().endswith('.pdf'):
        raise ValueError(f"Error: File is not a PDF ('.pdf' extension required).")

    try:
        # 3. Use LangChain's PyPDFLoader for robust text extraction
        loader = PyPDFLoader(file_path)
        
        # Load all pages as a list of Document objects
        pages = loader.load()
        print(f"Successfully loaded {len(pages)} pages.")

        # 4. Concatenate all page content into a single string
        full_text = "\n\n".join(page.page_content for page in pages)
        
        # Simple cleanup (optional, but helps with messy PDF parsing)
        full_text = re.sub(r'\s{2,}', ' ', full_text) # Replace multiple spaces/newlines with single space
        full_text = re.sub(r'(\n\s*){2,}', '\n\n', full_text) # Preserve paragraph breaks

        print("Text extraction complete.")
        return full_text
    
    except Exception as e:
        # Catch errors during PDF parsing
        raise RuntimeError(f"An error occurred during PDF text extraction: {e}")

def split_user_document(user_uploaded_text: str, source_name: str = "User TA") -> list[Document]:
    """
    Splits the raw text of the user-uploaded tenancy agreement into clause-level chunks.

    Args:
        user_uploaded_text: The raw string content of the user's document.
        source_name: A metadata tag to identify the source (e.g., the filename).

    Returns:
        A list of LangChain Document objects, one for each clause/chunk.
    """
    print(f"\n--- SPLITTING USER DOCUMENT: {source_name} ---")

    # The same structure-aware separators used for your Ideal Clauses (RAG 1)
    custom_separators = [
        "\n\n",
        r"\n\s*[A-Z]+\s+\d*\s*\.",
        r"\n\s*\d+\.\d*\s*",
        r"\n\s*\([a-zA-Z0-9]+\)\s*",
        "\n", " ", ""
    ]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        separators=custom_separators,
        is_separator_regex=True
    )

    # 1. Convert the single raw string into a list of Documents (one initial document)
    initial_document = [
        Document(page_content=user_uploaded_text, metadata={"source": source_name})
    ]

    # 2. Split the document based on the clause structure
    user_chunks = text_splitter.split_documents(initial_document)
    
    print(f"User TA split into {len(user_chunks)} clause-level chunks.")
    
    # Optional: Print a preview of the first few chunks
    for i, chunk in enumerate(user_chunks[:3]):
        print(f"  Chunk {i+1} (Length: {len(chunk.page_content)}): {chunk.page_content[:150].replace('\n', ' ')}...")
    
    return user_chunks

In [16]:
import json
import time
from typing import List, Dict
from langchain_core.documents import Document 
# Import the OpenAI library
from openai import OpenAI
import os 
import textwrap
from dotenv import load_dotenv

# --- LLM CONFIGURATION for OpenAI ---
# Recommended model for this task (fast and good at structured output)
LLM_MODEL = "gpt-4o-mini" 
MAX_RETRIES = 3
load_dotenv()  # Load environment variables from a .env file if present
# Get the API key from the environment variable
api_key_from_env = os.getenv("OPENAI_API_KEY")

if api_key_from_env:
    try:
        # Pass the key explicitly to ensure the client is initialized correctly
        client = OpenAI(api_key=api_key_from_env)
        print("OpenAI client initialized successfully from environment variable.")
    except Exception as e:
        print(f"FATAL ERROR: Failed to initialize OpenAI client even with key. Error: {e}")
        client = None
else:
    # If the key is not found, print a clear warning and set client to None
    print("OPENAI_API_KEY environment variable is NOT set.")
    client = None

def llm_compare_and_critique_openai(
    user_clause_text: str, 
    ideal_context_docs: List[Document]
) -> Dict:
    """
    Compares a user's tenancy clause against retrieved ideal context using the OpenAI API.
    
    The function uses structured JSON output enforcement for reliability.

    Args:
        user_clause_text: The content of the user's clause chunk.
        ideal_context_docs: A list of relevant ideal clauses retrieved from the RAG 1 Vector Store.
        
    Returns:
        A dictionary containing the feedback, risk level, and suggestions.
    """
    if not client:
        return {
            "clause_summary": "API Initialization Failed",
            "risk_level": "HIGH",
            "feedback": "OpenAI client is not configured correctly. Check API key.",
            "suggestion": "Set the OPENAI_API_KEY environment variable."
        }

    print(f"\n[CRITIQUE] Analyzing clause with GPT: {user_clause_text[:50]}...")

    # 1. FORMAT THE CONTEXT FOR THE LLM
    context_str = "\n---\n".join([doc.page_content for doc in ideal_context_docs])
    
    # 2. DEFINE SYSTEM INSTRUCTION
    # This sets the persona and output constraints.
    system_instruction = (
        "You are a world-class legal analyst specializing in residential tenancy agreements. "
        "Your task is to compare a provided 'User Clause' against 'Ideal Clause Examples' "
        "and generate structured, actionable feedback. Be concise, professional, and focus only on deviations or missing protections for the tenant. "
        "Your entire output MUST be a single, valid JSON object that strictly adheres to the provided JSON Schema."
    )

    # 3. DEFINE THE USER QUERY (THE CORE PROMPT)
    user_query = textwrap.dedent(f"""
        Please analyze the following 'User Clause' and compare it to the 'Ideal Clause Examples'.

        **USER CLAUSE TO CRITIQUE:**
        ---
        {user_clause_text}
        ---

        **IDEAL CLAUSE EXAMPLES (RAG Context):**
        ---
        {context_str}
        ---

        Based on your comparison, provide the analysis in the specified JSON format.
    """)

    # 4. DEFINE THE JSON SCHEMA
    response_schema = {
        "type": "object",
        "properties": {
            "clause_summary": {"type": "string", "description": "A brief (1-sentence) summary of the user clause's main topic."},
            "risk_level": {"type": "string", "enum": ["LOW", "MEDIUM", "HIGH"], "description": "The risk level (LOW, MEDIUM, or HIGH) for the tenant based on deviations from the ideal."},
            "feedback": {"type": "string", "description": "Specific, actionable criticism on what is missing or concerning in the User Clause."},
            "suggestion": {"type": "string", "description": "A brief sentence on how the user should attempt to modify the clause."}
        },
        "required": ["clause_summary", "risk_level", "feedback", "suggestion"]
    }


    # 5. EXECUTE API CALL WITH EXPONENTIAL BACKOFF
    for attempt in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model=LLM_MODEL,
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_query}
                ],
                # Use the response_format tool for guaranteed JSON output (GPT-4o/GPT-4 models)
                response_format={"type": "json_object"}, 
                # Note: The JSON structure is also implicitly constrained by the prompt and system instruction.
                temperature=0.0 # Use low temperature for analytical tasks
            )
            
            # Extract and parse the JSON response text
            # The entire output text should be a single JSON string
            json_text = response.choices[0].message.content
            parsed_json = json.loads(json_text)
            print(f"... Successful critique generated on attempt {attempt + 1}.")
            return parsed_json
            
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                wait_time = 2 ** attempt
                print(f"OpenAI API Error: {e}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"OpenAI API failed after {MAX_RETRIES} attempts.")
                return {
                    "clause_summary": "Analysis Failed",
                    "risk_level": "HIGH",
                    "feedback": f"Unable to generate critique after {MAX_RETRIES} attempts. Error: {e}",
                    "suggestion": "Check your API key, model permissions, and rate limits."
                }
    
    # Should be unreachable
    return {}

OpenAI client initialized successfully from environment variable.


In [ ]:
# Test Integrating LLM for RAG

# --- Configuration ---
FILE_NAME = 'Database Requirements.xlsx'
DATA_DIRECTORY = "./TA_template" 

# Initialize shared embedding model once
embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)

if __name__ == '__main__':
    # 1. BUILD THE TWO RAG SYSTEMS
    ideal_clauses_retriever = build_ideal_clauses_retriever(DATA_DIRECTORY)
    general_qa_retriever = build_general_qa_retriever(FILE_NAME)

    print("\n✅ BOTH RAG SYSTEMS ARE READY.")
    print("--------------------------------------------------")
    
    # --- PHASE 1: CLAUSE IDENTIFICATION & COMPARISON (RAG 1) ---
    # --- Placeholder: Replace this with the actual path to the user's uploaded file ---
    USER_UPLOADED_FILE_PATH = "./TA_sample/TA3.pdf" 
    # Note: In a real-world application, this path comes from your web/API framework after the user submits the file.

    try:
        # 1. LOAD & EXTRACT TEXT from the PDF
        full_user_document_text = load_and_extract_pdf_text(USER_UPLOADED_FILE_PATH)

        # 2. SPLIT the extracted text into clauses
        user_clause_chunks = split_user_document(
            full_user_document_text, 
            source_name=USER_UPLOADED_FILE_PATH
        )

        # 3. Loop through ALL user clauses for comparison (The RAG Core)
        feedback_report = []

        for user_clause in user_clause_chunks:
            # Use RAG 1 to retrieve the Ideal Clause context
            comparison_context = ideal_clauses_retriever.invoke(user_clause.page_content)
            
            # 4. LLM Call for Feedback (Simulated)
            feedback = llm_compare_and_critique_openai(user_clause.page_content, comparison_context)
            feedback_report.append(feedback)

        print("\n--- Phase 1 Complete: Analysis ready. ---")
        
    except (ValueError, RuntimeError) as e:
        # Handle the specific errors raised by the extraction function
        print(f"\nFATAL ERROR DURING FILE PROCESSING: {e}")
        # You would typically stop processing here and inform the user.